In [1]:
import json
import joblib
from tqdm import tqdm
from pathlib import Path

import pandas as pd
import numpy as np

import lightgbm as lgb
import catboost as cb

import h5py
from io import BytesIO
from PIL import Image

import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F

from timm import create_model

import albumentations as A
from albumentations.pytorch import ToTensorV2

from accelerate import Accelerator

from isic_helper import DotDict, get_folds

In [2]:
model_names = ["cb", "lgb", "resnet18", "efficientnet_b0"]
versions = ["v1", "v3", "v2",  "v1"]
paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-train" for model_name, version in zip(model_names, versions)]

weights = [1.0, 1.0, 0.8288018058494133, 0.4267574195330997]

In [3]:
id_column = "isic_id"
target_column = "target"
group_column = "patient_id"

In [4]:
def count_features(df, col):
    tmp = df[[id_column, group_column, col]].pivot_table(
        values=id_column, 
        index=group_column, 
        columns=col, 
        aggfunc="count", 
        fill_value=0)
    feature_cols = tmp.columns.tolist()
    tmp.reset_index(inplace=True)
    tmp.index.name = None
    df = df.merge(tmp, on=group_column, how="left")
    return df, feature_cols

def mean_features(df, col, val):
    tmp = df[[id_column, group_column, col, val]].pivot_table(
        values=val, 
        index=group_column, 
        columns=col, 
        aggfunc="mean", 
        fill_value=0)
    tmp.columns = [f"{c}_{val}_mean" for c in tmp.columns.tolist()]
    feature_cols = tmp.columns.tolist()
    tmp.reset_index(inplace=True)
    tmp.index.name = None
    df = df.merge(tmp, on=group_column, how="left")
    return df, feature_cols


def stat_features(df, group_cols, value_col, stats):
    tmp = df.groupby(group_cols)[value_col].agg(stats)
    tmp.columns = [f"{value_col}_{stat}" for stat in stats]
    tmp.reset_index(inplace=True)
    df = df.merge(tmp, on=group_cols, how="left")
    df[f"{value_col}_mean_diff"] = df[value_col] - df[f"{value_col}_mean"]
    return df


def feature_engineering(df):
    new_num_cols = []
    
    df["lesion_size_ratio"] = df["tbp_lv_minorAxisMM"] / df["clin_size_long_diam_mm"]
    new_num_cols += ["lesion_size_ratio"]
    
    df["lesion_distance"] = np.sqrt(df["tbp_lv_x"]**2 + df["tbp_lv_y"]**2 + df["tbp_lv_z"]**2)
    new_num_cols += ["lesion_distance"]
    
    df["hue_contrast"] = df["tbp_lv_H"] - df["tbp_lv_Hext"]
    df, feature_cols = mean_features(df, "anatom_site_general", "hue_contrast")
    new_num_cols += feature_cols
    
    df, feature_cols = count_features(df, "anatom_site_general")
    new_num_cols += feature_cols
    
    df["tbp_lv_A_diff"] =  df["tbp_lv_Aext"] - df["tbp_lv_A"]
    df = stat_features(df, ["patient_id", "tbp_lv_location"], "tbp_lv_A_diff", ["mean"])
    new_num_cols += ["tbp_lv_A_diff_mean_diff"]
    
    df["tbp_lv_B_diff"] =  df["tbp_lv_Bext"] - df["tbp_lv_B"]
    df = stat_features(df, ["patient_id", "tbp_lv_location"], "tbp_lv_B_diff", ["mean"])
    new_num_cols += ["tbp_lv_B_diff_mean_diff"]
    
    df["tbp_lv_L_diff"] =  df["tbp_lv_Lext"] - df["tbp_lv_L"]
    df = stat_features(df, ["patient_id", "tbp_lv_location"], "tbp_lv_L_diff", ["mean"])
    new_num_cols += ["tbp_lv_L_diff_mean_diff"]
    
    df["tbp_lv_L_std_diff"] =  df["tbp_lv_stdLExt"] - df["tbp_lv_stdL"]
    df = stat_features(df, ["patient_id", "tbp_lv_location"], "tbp_lv_L_std_diff", ["mean"])
    new_num_cols += ["tbp_lv_L_std_diff_mean_diff"]
    
    df["color_uniformity"] = df["tbp_lv_color_std_mean"] / df["tbp_lv_radial_color_std_max"]
    df, feature_cols = mean_features(df, "anatom_site_general", "color_uniformity")
    new_num_cols += feature_cols
    
    df["radius"] = np.cos(df["tbp_lv_symm_2axis_angle"]) * np.sqrt(df["tbp_lv_x"]**2 + df["tbp_lv_y"]**2 + df["tbp_lv_z"]**2)
    new_num_cols += ["radius"]
    
    return df, new_num_cols

In [5]:
def val_augment(image_size):
    transform = A.Compose([A.Resize(image_size, image_size), ToTensorV2()], p=1.0)
    return transform


class ISICDataset(Dataset):
    def __init__(self, metadata, images, augment, infer=False):
        self.metadata = metadata
        self.images = images
        self.augment = augment
        self.length = len(self.metadata)
        self.infer = infer

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        data = self.metadata.iloc[index]

        image = np.array(Image.open(BytesIO(self.images[data["isic_id"]][()])))
        image = self.augment(image=image)["image"]

        record = {"image": image}

        if not self.infer:
            target = data["target"]
            record["target"] = torch.tensor(target).float()

        return record


class ISICNet(nn.Module):
    def __init__(self, model_name, pretrained=True, infer=False):
        super(ISICNet, self).__init__()
        self.infer = infer
        self.model = create_model(
            model_name=model_name,
            pretrained=pretrained,
            in_chans=3,
            num_classes=0,
            global_pool="",
        )
        self.classifier = nn.Linear(self.model.num_features, 1)

        self.dropouts = nn.ModuleList([nn.Dropout(0.5) for i in range(5)])

    def forward(self, batch):
        image = batch["image"]
        image = image.float() / 255

        x = self.model(image)
        bs = len(image)
        pool = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)

        if self.training:
            logit = 0
            for i in range(len(self.dropouts)):
                logit += self.classifier(self.dropouts[i](pool))
            logit = logit / len(self.dropouts)
        else:
            logit = self.classifier(pool)
        return logit

In [6]:
INPUT_PATH = Path("../input/isic-2024-challenge/")

train_metadata = pd.read_csv(INPUT_PATH / "train-metadata.csv", low_memory=False)
test_metadata = pd.read_csv(INPUT_PATH / "test-metadata.csv")

folds_df = get_folds()
train_metadata = train_metadata.merge(folds_df, on=["isic_id", "patient_id"], how="inner")
print(f"Train data size: {train_metadata.shape}")
print(f"Test data size: {test_metadata.shape}")

train_metadata, new_num_cols = feature_engineering(train_metadata.copy())
test_metadata, _ = feature_engineering(test_metadata.copy())

test_images = h5py.File(INPUT_PATH / "test-image.hdf5", mode="r")

Train data size: (401059, 57)
Test data size: (3, 44)


In [7]:
def get_boosting_predictions(train, test, test_images, model_name, version, path):
    with open(path / f"{model_name}_{version}_encoder.joblib", "rb") as f:
        mixed_encoded_preprocessor = joblib.load(f)

    enc = mixed_encoded_preprocessor.fit(train)

    for col in mixed_encoded_preprocessor.feature_names_in_:
        if col not in test.columns:
            test[col] = np.nan

    X_test = enc.transform(test)

    columns_for_model = len(X_test.columns)
    print(f"Total number of columns: {columns_for_model}")

    with open(path / f"{model_name}_{version}_run_metadata.json", "r") as f:
        run_metadata = json.load(f)
        
    all_folds = np.unique(train["fold"])
    test_predictions_df = pd.DataFrame({id_column: test_metadata[id_column]})
    for fold in all_folds:
        model_filepath = path / f"models/{model_name}_{version}_fold_{fold}.txt"
        if "lgb" in model_name:
            model = lgb.Booster(model_file=model_filepath)
            test_predictions_df[f"fold_{fold}"] = model.predict(X_test, num_iteration=run_metadata["best_num_rounds"][f"fold_{fold}"])
        elif "cb" in model_name:
            model = cb.CatBoostClassifier(use_best_model=True)
            model.load_model(model_filepath)
            test_predictions_df[f"fold_{fold}"] = model.predict_proba(X_test)[:, -1]
    test_predictions_df[target_column] = test_predictions_df[[f"fold_{fold}" for fold in all_folds]].mean(axis=1)
    return test_predictions_df[[id_column, target_column]]


def get_dnn_predictions(train, test, test_images, model_name, version, path):
    with open(path / f"{model_name}_{version}_run_metadata.json", "r") as f:
        run_metadata = json.load(f)
        
    test_dataset = ISICDataset(
        test, test_images, augment=val_augment(run_metadata["params"]["image_size"]), infer=True
    )
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=run_metadata["params"]["val_batch_size"],
        shuffle=False,
        num_workers=2,
        drop_last=False,
        pin_memory=True,
    )
    
    all_folds = np.unique(train["fold"])
    test_predictions_df = pd.DataFrame({id_column: test_metadata[id_column]})
    for fold in all_folds:
        accelerator = Accelerator(
            mixed_precision=run_metadata["params"]["mixed_precision"],
        )
        
        model = ISICNet(model_name=model_name, pretrained=False, infer=True)
        model = model.to(accelerator.device)
        
        model, test_dataloader = accelerator.prepare(model, test_dataloader)
        model_filepath = path / f"models/fold_{fold}/epoch_{run_metadata['best_num_epochs'][f'fold_{fold}']}"
        accelerator.load_state(model_filepath)
        
        model.eval()
        test_preds = []
        with torch.no_grad():
            for batch in tqdm(test_dataloader, total=len(test_dataloader)):
                image0 = batch["image"].clone().detach()
                test_preds_batch = 0
                counter = 0
                with torch.no_grad():
                    outputs = model(batch)
                preds = torch.sigmoid(outputs)
                preds = accelerator.gather(preds)
                test_preds_batch += preds.data.cpu().numpy().reshape(-1)
                counter += 1
                if run_metadata["params"]["tta"]:
                    batch["image"] = torch.flip(image0, dims=[2])
                    with torch.no_grad():
                        outputs = model(batch)
                    preds = torch.sigmoid(outputs)
                    preds = accelerator.gather(preds)
                    test_preds_batch += preds.data.cpu().numpy().reshape(-1)
                    counter += 1

                    batch["image"] = torch.flip(image0, dims=[3])
                    with torch.no_grad():
                        outputs = model(batch)
                    preds = torch.sigmoid(outputs)
                    preds = accelerator.gather(preds)
                    test_preds_batch += preds.data.cpu().numpy().reshape(-1)
                    counter += 1

                    for k in [1, 2, 3]:
                        batch["image"] = torch.rot90(image0, k, dims=[2, 3])
                        with torch.no_grad():
                            outputs = model(batch)
                        preds = torch.sigmoid(outputs)
                        preds = accelerator.gather(preds)
                        test_preds_batch += preds.data.cpu().numpy().reshape(-1)
                        counter += 1
                test_preds_batch = test_preds_batch / counter
                test_preds.append(test_preds_batch)
        
        test_predictions_df[f"fold_{fold}"] = np.concatenate(test_preds)
    test_predictions_df[target_column] = test_predictions_df[[f"fold_{fold}" for fold in all_folds]].mean(axis=1)
    return test_predictions_df[[id_column, target_column]]

In [8]:
model_predict_function_topology = {
    "lgb": get_boosting_predictions,
    "cb": get_boosting_predictions,
    "resnet18": get_dnn_predictions,
    "efficientnet_b0": get_dnn_predictions
}

In [9]:
ensemble_preds = 0
previous_model_name = None
for idx, (model_name, version, path, weight) in enumerate(zip(model_names, versions, paths, weights)):
    print(f"Generating predictions for {model_name}_{version}")
    model_preds_df = model_predict_function_topology[model_name](train_metadata, test_metadata, test_images, model_name, version, Path(path))
    if idx == 0:
        ensemble_preds_df = model_preds_df.copy()
    else:
        ensemble_preds_df = ensemble_preds_df.merge(model_preds_df, on=id_column, how="inner", suffixes=(f"_{previous_model_name}", ""))
    ensemble_preds += ensemble_preds_df[target_column].rank(pct=True).values * weight
    previous_model_name = model_name
    print("\n")
ensemble_preds_df.rename(columns={target_column: f"{target_column}_{previous_model_name}"}, inplace=True)
ensemble_preds_df[target_column] = ensemble_preds

Generating predictions for cb_v1
Total number of columns: 60


Generating predictions for lgb_v3
Total number of columns: 60


Generating predictions for resnet18_v2


100%|██████████| 1/1 [00:00<00:00,  8.23it/s]




Generating predictions for efficientnet_b0_v1


100%|██████████| 1/1 [00:00<00:00,  6.50it/s]

In [10]:
ensemble_preds_df.head()

,isic_id,target_cb,target_lgb,target_resnet18,target_efficientnet_b0,target
0,ISIC_0015657,0.000182,0.000036,0.117095,0.032948,2.922226
1,ISIC_0015729,0.000023,0.000028,0.003008,0.002231,1.085186
2,ISIC_0015740,0.000113,0.000049,0.004665,0.005005,2.503706


In [11]:
ensemble_preds_df[target_column].describe()

count    3.000000
mean     2.170373
std      0.962815
min      1.085186
25%      1.794446
50%      2.503706
75%      2.712966
max      2.922226
Name: target, dtype: float64

In [12]:
ensemble_preds_df[[id_column, target_column]].head()

,isic_id,target
0,ISIC_0015657,2.922226
1,ISIC_0015729,1.085186
2,ISIC_0015740,2.503706


In [13]:
ensemble_preds_df[[id_column, target_column]].to_csv("submission.csv", index=False)